In [9]:
import pandas as pd
import numpy as np

import os
import argparse
import pickle
import joblib
import pdb
import warnings

In [36]:
cohort_path = "/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/"
# tasks = ['hospital_mortality', 'sepsis', 'LOS_7', 'readmission_30','aki_lab_abnormal_label','aki_lab_aki1_label','aki_lab_aki2_label','aki_lab_aki3_label','hyperkalemia_lab_abnormal_label','hyperkalemia_lab_mild_label','hyperkalemia_lab_moderate_label','hyperkalemia_lab_severe_label','hypoglycemia_lab_abnormal_label','hypoglycemia_lab_mild_label','hypoglycemia_lab_moderate_label','hypoglycemia_lab_severe_label','hyponatremia_lab_abnormal_label','hyponatremia_lab_mild_label','hyponatremia_lab_moderate_label','hyponatremia_lab_severe_label','neutropenia_lab_mild_label','neutropenia_lab_moderate_label','neutropenia_lab_severe_label','anemia_lab_abnormal_label','anemia_lab_mild_label','anemia_lab_moderate_label','anemia_lab_severe_label','thrombocytopenia_lab_abnormal_label','thrombocytopenia_lab_mild_label','thrombocytopenia_lab_moderate_label','thrombocytopenia_lab_severe_label']
tasks = ['hospital_mortality', 'sepsis', 'LOS_7', 'readmission_30','aki_lab_aki3_label','hyperkalemia_lab_severe_label','hypoglycemia_lab_severe_label','hyponatremia_lab_severe_label','neutropenia_lab_severe_label','anemia_lab_severe_label','thrombocytopenia_lab_severe_label']


In [11]:
def read_file(filename, columns=None, **kwargs):
    print(filename)
    load_extension = os.path.splitext(filename)[-1]
    if load_extension == ".parquet":
        return pd.read_parquet(filename, columns=columns,**kwargs)
    elif load_extension == ".csv":
        return pd.read_csv(filename, usecols=columns, **kwargs)

In [112]:
cohort = read_file(
    os.path.join(
        cohort_path,
        "cohort/cohort_split_no_nb.parquet"
    ),
    engine='pyarrow'
)


/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/cohort/cohort_split_no_nb.parquet


27409    -1
Name: age_days, dtype: Int64

In [56]:
print(cohort[['person_id', 'fold_id', 'adult_at_admission']].groupby(['adult_at_admission','fold_id']).count())

                            person_id
adult_at_admission fold_id           
0                  0            18623
                   test          4961
                   val           2852
1                  0           185370
                   test         38035
                   val          21406


In [98]:
ad_cohort = cohort.query("adult_at_admission==1")
ped_cohort = cohort.query("adult_at_admission==0")

In [110]:
ped_ages = [x.round(3) for x in list(ped_cohort['age_days']/365)]
print(np.median(ped_ages))
print(np.percentile(ped_ages, [25, 75]))
ad_ages = [x.round(3) for x in list(ad_cohort['age_days']/365)]
print(np.median(ad_ages))
print(np.percentile(ad_ages, [25, 75]))

5.608
[ 0.859 13.085]
56.093
[37.214 69.767]


In [15]:
ad_cohort.query("admission_year<=2019")[["person_id", "fold_id"]]#.groupby(["fold_id"]).count()

,person_id,fold_id
0,29936887,0
1,29936888,0
2,29936900,0
3,29936906,0
4,29936914,0
...,...,...
274191,43705036,0
274196,43705412,0
274216,43742869,0
274219,43743212,0


In [65]:
prev_df = pd.DataFrame()
for task in tasks:
    c_df = ped_cohort[["person_id",f"{task}"]]
    s = c_df.query(f"{task}==1")[task].sum()
    df = pd.DataFrame()
    df_dict = {"task":[task],
               "prevalence": [s/len(c_df)*100],
               "total_pos": [s],
               "total_pats": [len(c_df)]
              }
    df = pd.DataFrame(df_dict)
    prev_df = pd.concat((prev_df,df))
prev_df = prev_df.reset_index(drop=True)
with pd.option_context('display.max_rows', None,):
    print(prev_df)
        

                                 task  prevalence  total_pos  total_pats
0                  hospital_mortality    0.900287        238       26436
1                              sepsis    2.402028        635       26436
2                               LOS_7   19.473445       5148       26436
3                      readmission_30    6.056136       1601       26436
4                  aki_lab_aki3_label    1.066727        282       26436
5       hyperkalemia_lab_severe_label    1.577394        417       26436
6       hypoglycemia_lab_severe_label    1.316387        348       26436
7       hyponatremia_lab_severe_label    0.919201        243       26436
8        neutropenia_lab_severe_label    1.202905        318       26436
9             anemia_lab_severe_label    3.907550       1033       26436
10  thrombocytopenia_lab_severe_label    2.156151        570       26436


In [70]:
age_prev_df = pd.DataFrame()
for task in tasks:
    for aaa in [0]:#[0, 1]:
        for fold in ["0", "val", "test"]:
            c_df = cohort[["person_id",f"{task}", f"{task}_fold_id", "adult_at_admission"]].query(f"{task}_fold_id==@fold and adult_at_admission==@aaa")
            s = c_df.query(f"{task}==1")[task].sum()
            df = pd.DataFrame()
            df_dict = {"task":[task],
                       "age_group": ['pediatric'] if aaa==0 else ['adult'],
                       "fold":["train"] if fold == "0" else [fold],
                       "prevalence": [s/len(c_df)*100],
                       "total_pos": [s],
                       "total_pats": [len(c_df)]
                      }
            df = pd.DataFrame(df_dict)
            age_prev_df = pd.concat((age_prev_df,df))
age_prev_df = age_prev_df.reset_index(drop=True)
with pd.option_context('display.max_rows', None,):
    print(age_prev_df[['task','fold','prevalence','total_pos']])
        

                                 task   fold  prevalence  total_pos
0                  hospital_mortality  train    0.874287        161
1                  hospital_mortality    val    1.233698         35
2                  hospital_mortality   test    0.850547         42
3                              sepsis  train    2.648691        444
4                              sepsis    val    2.835249         74
5                              sepsis   test    2.551243        117
6                               LOS_7  train   18.859625       3473
7                               LOS_7    val   21.501586        610
8                               LOS_7   test   21.567436       1065
9                      readmission_30  train    5.991996       1093
10                     readmission_30    val    6.711889        188
11                     readmission_30   test    6.539955        320
12                 aki_lab_aki3_label  train    1.136861        208
13                 aki_lab_aki3_label    val    

In [47]:
print([c for c in cohort.columns])

['person_id', 'admit_date', 'discharge_date', 'admit_date_midnight', 'discharge_date_midnight', 'hospital_mortality', 'death_date', 'month_mortality', 'LOS_days', 'LOS_7', 'readmission_30', 'readmission_window', 'icu_admission', 'icu_start_datetime', 'age_in_years', 'age_group_x', 'race_eth', 'gender_concept_name', 'race_eth_raw', 'race_eth_gender', 'race_eth_age_group', 'race_eth_gender_age_group', 'race_eth_raw_gender', 'race_eth_raw_age_group', 'race_eth_raw_gender_age_group', 'prediction_id', 'fold_id', 'adult_at_admission', 'sepsis', 'sepsis_index_date', 'admission_year', 'discharge_year', 'hospital_mortality_fold_id', 'sepsis_fold_id', 'LOS_7_fold_id', 'readmission_30_fold_id', 'icu_admission_fold_id', 'birth_datetime', 'age_days', 'pediatric_age_group', 'age_group_y', 'sex', 'race', 'hyperkalemia_lab_max_potassium', 'hyperkalemia_lab_mild_measurement', 'hyperkalemia_lab_mild_measurement_datetime', 'hyperkalemia_lab_mild_label', 'hyperkalemia_lab_moderate_measurement', 'hyperkale

In [80]:
print(ad_cohort[['person_id', 'gender_concept_name']].groupby(['gender_concept_name']).count())
print(ad_cohort[['person_id', 'age_group_x']].groupby(['age_group_x']).count())
print(ad_cohort[['person_id', 'race_eth_raw']].groupby(['race_eth_raw']).count())

                     person_id
gender_concept_name           
FEMALE                  139785
MALE                    105013
No matching concept         13
             person_id
age_group_x           
[18-30)          28054
[30-45)          56152
[45-55)          32216
[55-65)          42577
[65-75)          44639
[75-91)          41173
                                           person_id
race_eth_raw                                        
American Indian or Alaska Native                 658
Asian                                          38570
Black or African American                       9519
Hispanic or Latino                             43689
Native Hawaiian or Other Pacific Islander       2803
Other                                          23073
White                                         126499


In [52]:
# ped_cohort[['person_id', 'pediatric_age_group']].groupby(['pediatric_age_group']).count()
for task in tasks:
    df = ped_cohort.query('pediatric_age_group=="term neonatal"')[['person_id', f'{task}']]
    print(df.groupby([f'{task}']).count())

                    person_id
hospital_mortality           
0                        3071
1                          81
        person_id
sepsis           
0            2786
1              69
       person_id
LOS_7           
0           2408
1            744
                person_id
readmission_30           
0                    2998
1                      71
                    person_id
aki_lab_aki3_label           
0                        3164
1                           7
                               person_id
hyperkalemia_lab_severe_label           
0                                   3041
1                                    115
                               person_id
hypoglycemia_lab_severe_label           
0                                   2817
1                                    161
                               person_id
hyponatremia_lab_severe_label           
0                                   3085
1                                     80
                        

In [21]:
cohort[['person_id', 'race_eth_raw']].groupby(['race_eth_raw']).count()

,person_id
race_eth_raw,
American Indian or Alaska Native,714
Asian,44324
Black or African American,10215
Hispanic or Latino,54334
Native Hawaiian or Other Pacific Islander,3191
Other,25158
White,136494


In [92]:
cohort = read_file(
    os.path.join(
        cohort_path,
        "cohort/cohort_split_no_nb_constrain_20.parquet"
    ),
    engine='pyarrow'
)

/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/cohort/cohort_split_no_nb_constrain_20.parquet


In [95]:
cohort.query("fold_id=='0' and adult_at_admission==0 and constrain==1")

,person_id,admit_date,discharge_date,admit_date_midnight,discharge_date_midnight,hospital_mortality,death_date,month_mortality,LOS_days,LOS_7,...,aki_lab_abnormal_label_fold_id,anemia_lab_mild_label_fold_id,anemia_lab_moderate_label_fold_id,anemia_lab_severe_label_fold_id,anemia_lab_abnormal_label_fold_id,thrombocytopenia_lab_mild_label_fold_id,thrombocytopenia_lab_moderate_label_fold_id,thrombocytopenia_lab_severe_label_fold_id,thrombocytopenia_lab_abnormal_label_fold_id,constrain
501,30044614,2017-07-24 21:49:00,2017-07-27 14:37:00,2017-07-24 23:59:00,2017-07-27 23:59:00,0,NaT,0,3,0,...,0,0,0,0,0,0,0,0,0,1
530,30046425,2017-12-16 19:18:00,2017-12-18 17:17:00,2017-12-16 23:59:00,2017-12-18 23:59:00,0,NaT,0,2,0,...,0,0,0,0,0,0,0,0,0,1
550,30047521,2015-11-14 09:03:00,2015-11-18 20:39:00,2015-11-14 23:59:00,2015-11-18 23:59:00,0,NaT,0,4,0,...,0,ignore,ignore,0,ignore,0,0,0,0,1
1156,30182596,2018-01-17 10:37:00,2018-01-18 14:58:00,2018-01-17 23:59:00,2018-01-18 23:59:00,0,NaT,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1200,30207619,2017-05-23 23:20:00,2017-05-25 10:20:00,2017-05-23 23:59:00,2017-05-25 23:59:00,0,NaT,0,2,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231267,43734122,2019-11-14 02:30:00,2019-11-17 12:07:00,2019-11-14 23:59:00,2019-11-17 23:59:00,0,NaT,0,3,0,...,0,0,0,0,0,0,0,0,0,1
231269,43734129,2019-12-30 23:08:00,2019-12-31 10:09:00,2019-12-30 23:59:00,2019-12-31 23:59:00,0,NaT,0,1,0,...,0,0,0,0,0,0,0,0,0,1
231271,43734133,2019-11-06 17:07:00,2019-11-23 17:12:00,2019-11-06 23:59:00,2019-11-23 23:59:00,0,NaT,0,17,1,...,0,ignore,ignore,0,ignore,ignore,0,0,ignore,1
231278,43734147,2019-12-03 16:31:00,2019-12-08 17:49:00,2019-12-03 23:59:00,2019-12-08 23:59:00,0,NaT,0,5,0,...,0,0,0,0,0,0,0,0,0,1
